In [64]:
import numpy as np
import pandas as pd
import talib as ta


In [49]:
df_all = pd.read_csv("Data/naive_prediction_data.csv")

In [50]:
df_all["30-day Bitcoin SMA"] = df_all['Value'].rolling(30).mean()
df_all["30-day Gold SMA"] = df_all['USD (PM)'].rolling(30).mean()


In [51]:
df_all["30-day Bitcoin rolling standard deviation"] = df_all['Value'].rolling(30).std()
df_all["30-day Gold rolling standard deviation"] = df_all['Value'].rolling(
    30).std()


In [52]:
df_all["30-day Bitcoin upper bolling bands"] = df_all["30-day Bitcoin SMA"] + \
    df_all["30-day Bitcoin rolling standard deviation"] * 2
df_all["30-day Bitcoin lower bolling bands"] = df_all["30-day Bitcoin SMA"] - \
    df_all["30-day Bitcoin rolling standard deviation"] * 2
df_all["30-day Gold upper bolling bands"] = df_all["30-day Gold SMA"] + \
    df_all["30-day Gold rolling standard deviation"] * 2
df_all["20-day Gold lower bolling bands"] = df_all["30-day Gold SMA"] - \
    df_all["30-day Gold rolling standard deviation"] * 2


In [53]:
def KAMA(price, n=10, pow1=2, pow2=30):
    ''' kama indicator '''
    ''' accepts pandas dataframe of prices '''

    abs_diff = abs(price - price.shift(1))

    ER_num = abs(price - price.shift(n))
    ER_den = abs_diff.rolling(n).sum()
    ER = ER_num / ER_den

    sc = (ER * (2.0 / (pow1 + 1) - 2.0 / (pow2 + 1.0)) + 2 / (pow2 + 1.0)) ** 2.0

    answer = np.zeros(sc.size)
    N = len(answer)
    first_value = True

    for i in range(N):
        if sc[i] != sc[i]:
            answer[i] = np.nan
        else:
            if first_value:
                answer[i] = price[i]
                first_value = False
            else:
                answer[i] = answer[i - 1] + sc[i] * (price[i] - answer[i - 1])
    return answer


In [54]:
df_all["Bitcoin KAMA"] = KAMA(df_all["Value"])
df_all["Gold KAMA"] = KAMA(df_all["USD (PM)"])

In [55]:
def PSY(x):
    return (x > 0).sum() / len(x)

In [56]:
df_all["Bitcoin PSY"] = df_all["Bitcoin % Change"].rolling(30).apply(PSY)
df_all["Gold PSY"] = df_all["Gold % Change"].rolling(30).apply(PSY)


In [57]:
_, _, df_all["Bitcoin MACD Histogram"] = ta.MACD(df_all["Value"])
_, _, df_all["Gold MACD Histogram"] = ta.MACD(df_all["USD (PM)"])


In [58]:
df_all["Bitcoin Momentum"] = ta.MOM(df_all["Value"])
df_all["Gold Momentum"] = ta.MOM(df_all["USD (PM)"])


In [59]:
df_all["Bitcoin RSI"] = ta.RSI(df_all["Value"])
df_all["Gold RSI"] = ta.RSI(df_all["USD (PM)"])


In [30]:
# df_all["Bitcoin TRIX"] = ta.TRIX(df_all["Value"])
# df_all["Gold TRIX"] = ta.TRIX(df_all["USD (PM)"])


In [60]:
df_all['RAT'] = df_all['Value'] / df_all['USD (PM)'] * 100

In [63]:
df_all.to_csv("Data/intermediate_data.csv", index=False)